In [42]:
import pandas as pd
import os
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import accuracy_score, recall_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

import wrangle as w
import explore as ex
import warnings
warnings.filterwarnings("ignore")

In [50]:
train,test=w.read_train_test()

In [51]:
len(train), len(test)

(4398, 1885)

1. Will be using adjusted_close as x_target.
2. Will Drop close, adjusted close, volume, rsi_14 for the y_target
3. May need to scale for some machine model.


In [52]:
train=train.dropna()
test=test.dropna()

In [53]:
train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4349 entries, 1999-03-16 to 2016-06-24
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   open            4349 non-null   float64
 1   high            4349 non-null   float64
 2   low             4349 non-null   float64
 3   close           4349 non-null   float64
 4   adjusted_close  4349 non-null   float64
 5   volume          4349 non-null   int64  
 6   sma_50          4349 non-null   float64
 7   rsi_14          4349 non-null   float64
dtypes: float64(7), int64(1)
memory usage: 305.8 KB


In [54]:
train_q = train.resample('Q').mean()
test_q= test.resample('Q').mean()

In [14]:
#create x /y targets
x_train= train.drop(columns=['volume','adjusted_close','close','rsi_14'])
y_train= train.adjusted_close
x_test= test.drop(columns=['volume','adjusted_close','close','rsi_14'])
y_test=test.adjusted_close

In [43]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

train_data = pd.concat([x_train, y_train], axis=1)
order=(1, 1, 1)

# ... (your code to fit the ARIMA model)
model = ARIMA(train[y_train.name], order=order)
model_fit = model.fit()

# Make predictions on the training set
y_train_pred = model_fit.predict()

# Calculate regression metrics
mse = mean_squared_error(y_train, y_train_pred)
mae = mean_absolute_error(y_train, y_train_pred)

# Print regression metrics
print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')


Mean Squared Error: 0.05506585494479958
Mean Absolute Error: 0.1215293666635869


### A lower MSE indicates better model performance, as it means the predicted values are closer to the actual values.
### A lower MAE indicates better model performance, as it means the absolute differences between the predicted and actual values are smaller.

In [44]:
import numpy as np
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LinearRegression, LassoLarsCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
alpha=0.05

def calculate_regression_results(X_train, y_train, X_test, y_test):
    # Calculate baseline (yhat)
    yhat = np.mean(y_train)

    # Initialize dataframe to store results
    results_df = pd.DataFrame(columns=['Model', 'Alpha', 'Degree', 'RMSE', 'R2'])

    # Ordinary Least Squares (OLS) regression
    ols = LinearRegression()
    ols.fit(X_train, y_train)
    ols_y_pred = ols.predict(X_test)
    ols_rmse = np.sqrt(mean_squared_error(y_test, ols_y_pred))
    ols_r2 = r2_score(y_test, ols_y_pred)
    results_df = results_df.append({'Model': 'OLS', 'Alpha': None, 'Degree': None, 'RMSE': ols_rmse, 'R2': ols_r2}, ignore_index=True)

    # LassoLARS regression with different alphas
    alphas = [0.1, 0.01, 0.001]
    for alpha in alphas:
        lasso = LassoLarsCV(cv=5, max_iter=10000, eps=0.1, normalize=True, precompute='auto')
        lasso.fit(X_train, y_train)
        lasso_y_pred = lasso.predict(X_test)
        lasso_rmse = np.sqrt(mean_squared_error(y_test, lasso_y_pred))
        lasso_r2 = r2_score(y_test, lasso_y_pred)
        results_df = results_df.append({'Model': 'LassoLARS', 'Alpha': alpha, 'Degree': None, 'RMSE': lasso_rmse, 'R2': lasso_r2}, ignore_index=True)

    # Polynomial regression with different degrees
    degrees = [1, 2, 3, 4]
    for degree in degrees:
        poly_features = PolynomialFeatures(degree=degree)
        X_train_poly = poly_features.fit_transform(X_train)
        X_test_poly = poly_features.transform(X_test)
        poly_reg = LinearRegression()
        poly_reg.fit(X_train_poly, y_train)
        poly_y_pred = poly_reg.predict(X_test_poly)
        poly_rmse = np.sqrt(mean_squared_error(y_test, poly_y_pred))
        poly_r2 = r2_score(y_test, poly_y_pred)
        results_df = results_df.append({'Model': 'Polynomial Regression', 'Alpha': None, 'Degree': degree, 'RMSE': poly_rmse, 'R2': poly_r2}, ignore_index=True)

    return results_df

In [45]:
calculate_regression_results(x_train, y_train, x_test, y_test)

,Model,Alpha,Degree,RMSE,R2
0,OLS,NaN,None,0.832788,0.999599
1,LassoLARS,0.100,None,1.261817,0.999079
2,LassoLARS,0.010,None,1.261817,0.999079
3,LassoLARS,0.001,None,1.261817,0.999079
4,Polynomial Regression,NaN,1,0.832788,0.999599
5,Polynomial Regression,NaN,2,1.545307,0.998618
6,Polynomial Regression,NaN,3,33.677026,0.343724
7,Polynomial Regression,NaN,4,1228.078453,-871.712785
